In [2]:
%load_ext autoreload
%autoreload 2
from markov import *

# Making of the add_string() method

In [41]:
model = MarkovModel(5)
model.add_string("CS 477 rocks")
#model.get_prefixs()
#model.get_prefix_count()
print(model)

CS 47 {'7': 1}
S 477 {' ': 1}
 477  {'r': 1}
477 r {'o': 1}
77 ro {'c': 1}
7 roc {'k': 1}
 rock {'s': 1}
rocks {'C': 1}
ocksC {'S': 1}
cksCS {' ': 1}
ksCS  {'4': 1}
sCS 4 {'7': 1}
This is a Markov model with prefix length 5.


### Checking actual data

In [34]:
model = MarkovModel(5)
model.load_file_lines("text/spongebobquotes.txt")
print(model.get_prefix_count())
print(model.get_prefix(' you '))
#print(model)

4032
{'s': 4, 'm': 1, 't': 4, 'l': 1, 'j': 1, 'w': 1, 'a': 4, 'b': 1, 'h': 1, 'c': 4, 'k': 1, 'e': 1, 'g': 1}


In [14]:
len(model.markov_dict[' you '])

13

`{'s': 4, 'm': 1, 't': 4, 'l': 1, 'j': 1, 'w': 1, 'a': 4, 'b': 1, 'h': 1, 'c': 4, 'k': 1, 'e': 1, 'g': 1}`

Should have 4032 unique prefixs;

`' you '` will have all the characters above in the dict

# Computing Probabilities (10 Pts)

"aaacbaaaacabacbbcabcccbccaaac" <br>
probablity of "aacabaa" = -3.91

In [111]:
m = MarkovModel(3)
m.add_string("aaacbaaaacabacbbcabcccbccaaac")
#print(m)
print("probablity: {:.2f}".format(m.get_log_probability("aacabaa", debug=True)))

prefix: aac. character a. np: 3. npc: 2. curr prob: -0.6931471805599453.
prefix: aca. character b. np: 2. npc: 1. curr prob: -0.916290731874155.
prefix: cab. character a. np: 2. npc: 1. curr prob: -0.916290731874155.
prefix: aba. character a. np: 1. npc: 0. curr prob: -1.3862943611198906.
probablity: -3.91


In [116]:
model = MarkovModel(6)
model.load_file_lines("text/spongebobquotes.txt")
#print(model.get_unique_chars())
print("probablity: {:.1f}".format(model.get_log_probability("I'm ready, I'm ready")))

probablity: -40.5


probablity should be "-40.5"

In [110]:
print("probablity: {:.1f}".format(model.get_log_probability("Artificial intelligence")))

probablity: -69.6


probablity should be "-69.6"

# 2016 US Presidental Debate (10 Pts)

In [117]:
K = 5
trump = MarkovModel(K)
trump.load_file("text/2016Debates/trump1.txt")
trump.load_file("text/2016Debates/trump2.txt")
clinton = MarkovModel(K)
clinton.load_file("text/2016Debates/clinton1.txt")
clinton.load_file("text/2016Debates/clinton2.txt")


In [120]:
#example of how to load all of the segments from Trump as strings
for i in range(40):
    fin = open("text/2016Debates/trump3-{:02d}.txt".format(i), encoding="utf8")
    s = fin.read() # This holds the text as a string
    #print(s)
    fin.close()
    #TODO:

In [121]:
'''train clinton model on clinton quotes from the first two debates
train trump model on trump quotes from the first two debates
 
for quote in third clinton debate:
    prob1 = log probability of quote given clinton model
    prob2 = log probability of quote given trump model
    if prob2 > prob1:
        correct++
    else
        incorrect++
'''

'train clinton model on clinton quotes from the first two debates\ntrain trump model on trump quotes from the first two debates\n \nfor quote in third clinton debate:\n    prob1 = log probability of quote given clinton model\n    prob2 = log probability of quote given trump model\n    if prob2 > prob1:\n        correct++\n    else\n        incorrect++\n'